In [78]:
import os
import numpy as np
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup

In [79]:
def get_bvflow(dd):
    
    inflows = ['Deer Creek Penstock #1 Flow', 
               'Deer Creek Penstock #2 Flow', 
               'Deer Creek Spillway Flow']
    
    diversions = ['Deer Creek Salt Lake Aqueduct Flow']

    # Can set an offset/bias correct here
    bvflow = 0
    
    print('%s\n'%datadict['Deer Creek Penstock #1 Flow']['time'][-1])

    for k in dd.keys():

        if k in inflows:
            bvflow += dd[k]['value'][-1]
            print('%s:\t\t+%.0f cfs'%(k, dd[k]['value'][-1]))

        elif k in diversions:
            bvflow -= dd[k]['value'][-1]
            print('%s:\t-%.0f cfs'%(k, dd[k]['value'][-1]))

    # Try to find a source for the actual power plant diversion in realtime...
    # for now assume max of 515 cfs
    maxpower = 515
    powergen = maxpower if bvflow > maxpower else bvflow        
    bvflow -= powergen

    print('Deer Creek Power Generation:\t\t-%.0f cfs'%powergen)
    
    print('\nEstimated BV Flow:\t\t\t%.0f cfs'%bvflow)
    
    return round(bvflow, 2)

In [80]:
df = '/uufs/chpc.utah.edu/common/home/u1070830/rio_project/data/provoBV.npy'
new = False if os.path.isfile(df) else True

r = requests.get('https://www.prwua.org/water-operations/live-SCADA-system-data.php')
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find(id='livescada')
rows = table.find_all('tr')

In [101]:
if new:
    datadict = {}
else:
    datadict = np.load(df)[()]

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]

    sid = cols[0]
    site = cols[3]
    
    time = datetime.strptime(cols[1], '%m/%d/%Y %I:%M:%S %p')
    
    value = float(cols[2])
    valtype = site.split(' ')[-1].lower()

    if new:
        datadict[site] = {'sid':sid, 'type':valtype, 'time':[time], 'value':[value]}
        
    else:
        datadict[site]['time'].append(time)
        datadict[site]['value'].append(value)
        
if new:
    datadict['Provo River Bridal Veil Flow'] = {'sid':'BV_EST', 'type':'flow', 
                                                'time':[datadict['Deer Creek Penstock #1 Flow']['time'][-1]],
                                                'value':[get_bvflow(datadict)]}
else:
    datadict['Provo River Bridal Veil Flow']['time'].append(datadict['Deer Creek Penstock #1 Flow']['time'][-1])
    datadict['Provo River Bridal Veil Flow']['value'].append(get_bvflow(datadict))
        
# np.save(df, datadict)

2019-05-30 12:09:57

Deer Creek Salt Lake Aqueduct Flow:	-30 cfs
Deer Creek Penstock #1 Flow:		+516 cfs
Deer Creek Penstock #2 Flow:		+524 cfs
Deer Creek Spillway Flow:		+0 cfs
Deer Creek Power Generation:		-515 cfs

Estimated BV Flow:			495 cfs


In [82]:
with open('/uufs/chpc.utah.edu/common/home/u1070830/rio_project/bvlog.txt', 'r+') as rfp:
    with open('/uufs/chpc.utah.edu/common/home/u1070830/public_html/rivers/provo/bvlog.csv', 'w+') as wfp:

        csvdata = rfp.readlines()
        lines = len(csvdata)
        maxlines = 288 # 12x hour * 24 hours
        
        if lines <= maxlines:
                [wfp.write(line) for line in csvdata]
                
        else:
            wfp.write(csvdata[0])
            for line in csvdata[-maxlines:]:
                wfp.write(line)
                